In [6]:
%matplotlib inline

import operator
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import spacy
import time
from datetime import datetime
from spacy.symbols import ORTH, LEMMA, POS

In [7]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.tweets={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        searchTime=datetime.now()
        # get 1/4 of the corpus or 20 tweets, which ever is larger.
        search_size=20
        if corpus_size/4 > search_size:
            search_size = corpus_size/4
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",count=search_size)
            print("got "+str(len(new_tweets))+" candidate tweets")
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt,searchTime,term)
            print("....."+str(len(self.tweets))+" tweets total")
            time.sleep(5)
                
    def addTweet(self,tweet,searchTime,term="",count=0):
        id = tweet['id_str']
        if id not in self.tweets.keys():
            self.tweets[id]={}
            self.tweets[id]['tweet']=tweet
            self.tweets[id]['count']=0
            self.tweets[id]['searchTime']=searchTime
            self.tweets[id]['searchTerm']=term
        self.tweets[id]['count'] = self.tweets[id]['count'] +1
        
    def getTweet(self,id):
        if id in self.tweets:
            return self.tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.tweets[id]['count']
    
    def countTweets(self):
        return len(self.tweets)
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        for t,entry in self.tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.tweets.keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.tweets)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.tweets=incontents
        
    def getSearchTerm(self,id):
        return self.tweets[id]['searchTerm']
    
    def getSearchTime(self,id):
        return self.tweets[id]['searchTime']
    
    def getText(self,id):
        tweet = self.getTweet(id)
        text=tweet['full_text']
        if 'retweeted_status'in tweet:
            original = tweet['retweeted_status']
            text=original['full_text']
        return text
                
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
   
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
 
    def getCodes(self,id):
        tweet=self.getTweet(id)
        return tweet['codes']
    
    # NEW -ROUTINE TO GET PROFILE
    def getCodeProfile(self):
        summary={}
        for id in self.tweets.keys():
            tweet=self.getTweet(id)
            if 'codes' in tweet:
                for code in tweet['codes']:
                    if code not in summary:
                            summary[code] =0
                    summary[code]=summary[code]+1
        sortedsummary = sorted(summary.items(),key=operator.itemgetter(0),reverse=True)
        return sortedsummary

In [8]:
consumer_key='D2L4YZ2YrO1PMix7uKUK63b8H'
consumer_secret='losRw9T8zb6VT3TEJ9JHmmhAmn1GXKVj30dkiMv9vjhXuiWek9'
access_token='15283934-iggs1hiZAPI2o5sfHWMfjumTF7SvytHPjpPRGf3I6'
access_secret='bOvqssxS97PGPwXHQZxk83KtAcDyLhRLgdQaokCdVvwFi'

In [9]:
from tweepy import OAuthHandler

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [10]:
t1 = api.search("vaping",lang="en",count=10)

In [11]:
t1 = [t._json for t in t1]

In [12]:
t1ids = [t['id_str'] for t in t1]

In [13]:
t1ids

['1015238394841092096',
 '1015238354819051521',
 '1015237987100225538',
 '1015237928266723329',
 '1015237916598161408',
 '1015237872033595393',
 '1015237768090476550',
 '1015237513835876353',
 '1015237262181879809']

In [14]:
t2 = api.search("vaping",lang="en",count=10)

In [15]:
t2 = [t._json for t in t2]

In [16]:
t2ids = [t['id_str'] for t in t2]

In [17]:
t2ids

['1015238394841092096',
 '1015238354819051521',
 '1015237987100225538',
 '1015237928266723329',
 '1015237916598161408',
 '1015237872033595393',
 '1015237768090476550',
 '1015237513835876353',
 '1015237262181879809']

In [18]:
set(t1ids).intersection(t2ids)

{'1015237262181879809',
 '1015237513835876353',
 '1015237768090476550',
 '1015237872033595393',
 '1015237916598161408',
 '1015237928266723329',
 '1015237987100225538',
 '1015238354819051521',
 '1015238394841092096'}

In [19]:
t3 = api.search("vaping",lang="en",count=50)

In [20]:
t4 = api.search("vaping",lang="en",count=50)

In [21]:
t3 = [t._json for t in t3]

In [22]:
t3ids = [t['id_str'] for t in t3]

In [23]:
t4 = [t._json for t in t4]

In [24]:
t4ids = [t['id_str'] for t in t4]

In [25]:
len(set(t3ids).intersection(t4ids))

45

In [26]:
t5 = api.search("vaping",lang="en",count=50)
t5ids= [t._json['id_str'] for t in t5]

In [27]:
len(set(t3ids).intersection(t5ids))

45

In [28]:
t6= api.search("vaping",lang="en",count=50)
t6ids= [t._json['id_str'] for t in t6]

In [29]:
len(set(t3ids).intersection(t6ids))

45

In [30]:
lists =[]
for i in range(10):
    res = api.search("vaping",lang="en",count=50)
    resids=[t._json['id_str'] for t in res]
    lists.append(resids)
    #time.sleep(10)

for i in range(1,10):
    prior = lists[i-1]
    cur=lists[i]
    newids = set(prior).intersection(cur)
    unique = len(cur)-len(newids)
    print(unique)

0
0
0
0
0
0
0
0
0


In [31]:
t4 = api.search("vaping",lang="en",count=500)

In [32]:
t4ids= [t._json['id_str'] for t in t4]

In [33]:
len(t4ids)

89

In [34]:
t6 = api.search("vaping",lang="en",count=1000)

In [35]:
len(t6)

89

In [36]:
t7 = api.search("vaping",lang="en",count=1000)

In [37]:
t6ids= [t._json['id_str'] for t in t6]
t7ids= [t._json['id_str'] for t in t7]

In [38]:
len(set(t6ids).intersection(t7ids))

89

In [39]:
t8 = api.search("vaping",lang="en",count=1000)

In [40]:
t8ids= [t._json['id_str'] for t in t8]
len(set(t6ids).intersection(t8ids))

89

In [41]:
len(t8ids)

89

In [42]:
t9 = api.search("vaping",lang="en",count=1000)
t9ids= [t._json['id_str'] for t in t9]
len(set(t8ids).intersection(t9ids))

89

In [43]:
time

<module 'time' (built-in)>

In [44]:
import datetime
datetime.datetime.now()

datetime.datetime(2018, 7, 6, 10, 18, 25, 956535)

In [45]:
print(datetime.datetime.now())
t10 = api.search("vaping",lang="en",count=1000)
t10ids= [t._json['id_str'] for t in t10]

2018-07-06 10:18:26.020425


In [46]:
print(datetime.datetime.now())
t11 = api.search("vaping",lang="en",count=1000)
t11ids= [t._json['id_str'] for t in t11]
len(set(t10ids).intersection(t11ids))

2018-07-06 10:18:26.500067


89

In [47]:
print(datetime.datetime.now())
tstart = api.search("vaping",lang="en",count=1000)
tstartids= [t._json['id_str'] for t in tstart]
time.sleep(300)
print(datetime.datetime.now())
tend= api.search("vaping",lang="en",count=1000)
tendids= [t._json['id_str'] for t in tend]
len(set(tstartids).intersection(tendids))

2018-07-06 10:18:27.134713
2018-07-06 10:23:27.595351


85

ok. 85% overlap after a 5 minute break.

In [48]:
print(datetime.datetime.now())
tstart = api.search("vaping",lang="en",count=1000)
tstartids= [t._json['id_str'] for t in tstart]
time.sleep(600)
print(datetime.datetime.now())
tend= api.search("vaping",lang="en",count=1000)
tendids= [t._json['id_str'] for t in tend]
len(set(tstartids).intersection(tendids))

2018-07-06 10:23:28.237876
2018-07-06 10:33:28.707178


75

75% overlap after 10 minutes

In [49]:
print(datetime.datetime.now())
tstart = api.search("vaping",lang="en",count=1000)
tstartids= [t._json['id_str'] for t in tstart]
time.sleep(900)
print(datetime.datetime.now())
tend= api.search("vaping",lang="en",count=1000)
tendids= [t._json['id_str'] for t in tend]
len(set(tstartids).intersection(tendids))

2018-07-06 10:41:31.254657
2018-07-06 10:56:31.908626


66

after 15 minutes 66% overlap.

In [52]:
print(datetime.datetime.now())
tstart = api.search("vaping",lang="en",count=1000)
tstartids= [t._json['id_str'] for t in tstart]
time.sleep(120)
print(datetime.datetime.now())
tend= api.search("vaping",lang="en",count=1000)
tendids= [t._json['id_str'] for t in tend]
len(set(tstartids).intersection(tendids))

2018-07-06 10:57:54.620915
2018-07-06 10:59:55.186456


87

In [53]:
print(datetime.datetime.now())
tstart = api.search("vaping",lang="en",count=100)
tstartids= [t._json['id_str'] for t in tstart]
time.sleep(1200)
print(datetime.datetime.now())
tend= api.search("vaping",lang="en",count=100)
tendids= [t._json['id_str'] for t in tend]
len(set(tstartids).intersection(tendids))

2018-07-06 11:00:16.000828
2018-07-06 11:20:16.528071


52

In [54]:
print(datetime.datetime.now())
tstart = api.search("vaping",lang="en",count=100)
tstartids= [t._json['id_str'] for t in tstart]
time.sleep(1200)
print(datetime.datetime.now())
tend= api.search("vaping",lang="en",count=100)
tendids= [t._json['id_str'] for t in tend]
len(set(tstartids).intersection(tendids))

2018-07-06 11:28:50.498127
2018-07-06 11:48:51.402180


45

In [55]:
len(tstart)

86

In [56]:
len(tend)

93